In [ ]:
import threading

def is_prime(n):
    if n < 2:
        return False
    if n == 2:
        return True
    if n % 2 == 0:
        return False
    for i in range(3, int(n**0.5) + 1, 2):
        if n % i == 0:
            return False
    return True

def check_primes_in_range(start, end, result_list):
    for number in range(start, end + 1):
        if is_prime(number):
            result_list.append(number)

def threaded_prime_checker(start, end, num_threads):
    threads = []
    results = []
    result_lock = threading.Lock()

    # Calculate the size of each chunk
    total_numbers = end - start + 1
    chunk_size = total_numbers // num_threads

    # Shared result list for thread-safe appending
    shared_results = []

    def thread_task(s, e):
        local_primes = []
        for num in range(s, e + 1):
            if is_prime(num):
                local_primes.append(num)
        # Lock to safely append results
        with result_lock:
            shared_results.extend(local_primes)

    # Create and start threads
    for i in range(num_threads):
        chunk_start = start + i * chunk_size
        # Last chunk takes any leftover numbers
        chunk_end = start + (i + 1) * chunk_size - 1 if i != num_threads - 1 else end
        t = threading.Thread(target=thread_task, args=(chunk_start, chunk_end))
        threads.append(t)
        t.start()

    # Wait for all threads to finish
    for t in threads:
        t.join()

    # Sort the primes since threads may append out of order
    shared_results.sort()
    return shared_results

#


In [ ]:
import threading
from collections import Counter

def count_words_in_lines(lines, result_counter, lock):
    local_counter = Counter()
    for line in lines:
        words = line.strip().split()
        local_counter.update(words)
    # Safely update the shared counter
    with lock:
        result_counter.update(local_counter)

def threaded_word_count(file_path, num_threads):
    # Read all lines from the file
    with open(file_path, 'r') as f:
        lines = f.readlines()

    total_lines = len(lines)
    chunk_size = total_lines // num_threads
    threads = []
    result_counter = Counter()
    lock = threading.Lock()

    for i in range(num_threads):
        start = i * chunk_size
        # Last thread takes remaining lines
        end = (i + 1) * chunk_size if i != num_threads - 1 else total_lines
        thread_lines = lines[start:end]
        t = threading.Thread(target=count_words_in_lines, args=(thread_lines, result_counter, lock))
        threads.append(t)
        t.start()

    for t in threads:
        t.join()

    return result_counter

if __name__ == "__main__":
    file_path = "large_text_file.txt"  # Replace with your file path
    threads_count = 4

    word_counts = threaded_word_count(file_path, threads_count)

    print("Word occurrences:")
    for word, count in word_counts.most_common():
        print(f"{word}: {count}")
